In [2]:
# Import libraries
import pandas as pd
import numpy as np
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy.stats import norm
import streamlit as st
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import cross_validation
import plotly.graph_objects as go
from datetime import timedelta
from sklearn.metrics import r2_score


In [3]:
# Load & clean data for Great Britain
d2023 = pd.read_excel('monthly_hourly_load_values_2023.xlsx')
d2024 = pd.read_excel('monthly_hourly_load_values_2024.xlsx')

c:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# Append the data
data =  pd.concat([d2023, d2024], ignore_index=True)

# Filter for Great Britain
data = data[data['CountryCode'] == 'GB']

data['Date'] = data['DateUTC']

# Convert the 'Date' column to a consistent datetime format
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)

# Reset index to turn 'Date' back into a column before renaming
GB = data.reset_index()

# Rename the columns in daily_untrade
GB = GB.rename(columns={
    'Date': 'ds',
    'Value': 'y'
})

# Remove unnecessary columns
GB = GB[["ds", "y"]]

# Filter data for May 23 2023 onwards
GB = GB[810:]

# # Sort Datetime
# GB = GB.sort_values(by='ds')

# Check for any parsing errors
# GB.to_excel("GB.xlsx", index=False)
# print(GB['ds'])
# type(GB['ds'])
# print(GB['y'])
# type(GB['ds'])


In [ ]:
# Plot the electricity demand
fig = go.Figure()
fig.add_trace(go.Scatter(x=GB['ds'], y=GB['y'],
                    mode='lines',
                    name='lines'))

# Customize layout for better visualization
fig.update_layout(
    template="plotly_dark",  # Optional: Dark theme
    yaxis=dict(
       autorange = True,
       fixedrange= False
   )
)

# fig.show()

In [48]:
# Model and Prediction

periods=24
m = Prophet(changepoint_prior_scale=0.5, 
            seasonality_prior_scale = 10,
            seasonality_mode = "multiplicative",
            daily_seasonality=True,
            yearly_seasonality=True
            ).fit(GB)
future = m.make_future_dataframe(periods, freq='h')
fcst = m.predict(future)


09:19:24 - cmdstanpy - INFO - Chain [1] start processing
09:19:36 - cmdstanpy - INFO - Chain [1] done processing


In [94]:
# Plot the forecasts
fig = plot_plotly(m, fcst)

# Customize layout for better visualization
fig.update_layout(
    title="Great Britain Hourly Energy Demand Forecasts",
    xaxis_title="Date",
    yaxis_title="Demand",
    template="plotly_dark",  # Dark theme
    yaxis=dict(
        autorange=True,
        fixedrange=False
    ),
    xaxis=dict(
        rangeselector=dict(visible=False)  # Disable timeframe buttons
    )
)

# Update scatter plot markers for demand values to be white
fig.update_traces(
    marker=dict(color="mintcream"),
    selector=dict(mode="markers", type="scatter")
)


In [97]:
# Compute the R-squared
metric_df = fcst.set_index('ds')[['yhat']].join(GB.set_index('ds').y).reset_index()
metric_df.tail()
metric_df.dropna(inplace=True)
r2 = r2_score(metric_df.y, metric_df.yhat)
print('R-Squared = {:,.2%}'.format(r2))


R-Squared = 84.81%


In [109]:
# Average hourly forecast
absolute = fcst.tail(periods)
absolute_sum = np.average(absolute["yhat"])
print('Average hourly forecast is {:,.2f}'.format(absolute_sum))

Average hourly forecast is 728.80


In [6]:
# # Define cross-validation parameters
# initial = '336 hours'   # Two weeks for initial training period
# period = '12 hours'     # Evaluate every 12 hours
# horizon = '24 hours'    # Forecast one day ahead

# df_cv = cross_validation(m, initial=initial, period=period, horizon=horizon)
# df_cv.head()

# from prophet.diagnostics import performance_metrics
# df_p = performance_metrics(df_cv)
# df_p.head()


  0%|          | 0/780 [00:00<?, ?it/s]

19:15:20 - cmdstanpy - INFO - Chain [1] start processing
19:15:20 - cmdstanpy - INFO - Chain [1] done processing
19:15:20 - cmdstanpy - INFO - Chain [1] start processing
19:15:20 - cmdstanpy - INFO - Chain [1] done processing
19:15:21 - cmdstanpy - INFO - Chain [1] start processing
19:15:21 - cmdstanpy - INFO - Chain [1] done processing
19:15:21 - cmdstanpy - INFO - Chain [1] start processing
19:15:21 - cmdstanpy - INFO - Chain [1] done processing
19:15:21 - cmdstanpy - INFO - Chain [1] start processing
19:15:21 - cmdstanpy - INFO - Chain [1] done processing
19:15:21 - cmdstanpy - INFO - Chain [1] start processing
19:15:21 - cmdstanpy - INFO - Chain [1] done processing
19:15:21 - cmdstanpy - INFO - Chain [1] start processing
19:15:21 - cmdstanpy - INFO - Chain [1] done processing
19:15:22 - cmdstanpy - INFO - Chain [1] start processing
19:15:22 - cmdstanpy - INFO - Chain [1] done processing
19:15:22 - cmdstanpy - INFO - Chain [1] start processing
19:15:22 - cmdstanpy - INFO - Chain [1]

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,0 days 03:00:00,5284.831925,72.696849,56.529506,0.078689,0.060655,0.078086,0.766080
1,0 days 04:00:00,5039.710279,70.990917,55.515735,0.079733,0.061684,0.079070,0.772752
2,0 days 05:00:00,5300.081164,72.801656,57.852473,0.083558,0.064396,0.082189,0.749456
3,0 days 06:00:00,6539.927615,80.869819,63.549632,0.091259,0.069215,0.088131,0.718668
4,0 days 07:00:00,11499.019690,107.233482,81.838249,0.112675,0.079739,0.107058,0.634583
